In [2]:
from movie_recommender import MovieRecommender
import timeit
import pandas as pd

In [2]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')

print(movie_rec.ratings_df['user_id'].nunique())

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
Read dataframes in 31.031014400003187ms
69878


In [3]:
# replace pipe character with space
movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')
rating_counts = movie_rec.ratings_df.groupby('movie_id').agg('rating').count()
rating_counts.name = 'num_ratings'
rating_avg = movie_rec.ratings_df.groupby('movie_id').agg('rating').mean()
rating_avg.name = 'avg_rating'
most_rated = rating_counts.sort_values(ascending=False).head(1000)
most_rated = pd.merge(most_rated, movie_rec.movies_df, on="movie_id")
most_rated = pd.merge(rating_avg, most_rated, on="movie_id")
most_rated
most_rated.sort_values(by='num_ratings', ascending=False, inplace=True)


C:\Users\john.french\AppData\Local\Temp\ipykernel_22428\2230656454.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')


In [4]:
for m in most_rated.index[:100]:
    print(movie_rec.get_movie_title(m))

Pulp Fiction
Forrest Gump
Silence of the Lambs, The
Jurassic Park
Shawshank Redemption, The
Braveheart
Fugitive, The
Terminator 2: Judgment Day
Star Wars: Episode IV - A New Hope (a.k.a. Star Wars)
Apollo 13
Batman
Toy Story
Independence Day (a.k.a. ID4)
Dances with Wolves
Schindler's List
True Lies
Star Wars: Episode VI - Return of the Jedi
12 Monkeys (Twelve Monkeys)
Usual Suspects, The
Fargo
Speed
Aladdin
Matrix, The
Star Wars: Episode V - The Empire Strikes Back
Seven (a.k.a. Se7en)
American Beauty
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)
Back to the Future
Mission: Impossible
Ace Ventura: Pet Detective
Lion King, The
Beauty and the Beast
Godfather, The
Die Hard: With a Vengeance
Sixth Sense, The
Batman Forever
Pretty Woman
Mrs. Doubtfire
Mask, The
Stargate
Saving Private Ryan
Babe
Twister
Men in Black
Clear and Present Danger
Rock, The
Dumb & Dumber
E.T. the Extra-Terrestrial
Terminator, The
Ghost
Sleepless in Seattle
GoldenEye
Groundhog Day
Four Wed

In [5]:
movie_rec_meta = list()

# print(movie_rec.df)
for m in most_rated.index[:100]:
        
    u = int(movie_rec.ratings_df['user_id'].max()+1)

    movie_recs = dict()
    movie_recs['movie_id'] = m
    movie_rec.new_add_movie_rating(movie_id=m, user_id=u, rating=5)

    movie_rec.refit_model()
    recs = movie_rec.get_user_based_recommendation(u, m)
    movie_recs['similar_movie_ids'] = recs[:3]
    movie_rec.flush_user_data()

    movie_rec_meta.append(movie_recs)
movie_rec_meta_df = pd.DataFrame(movie_rec_meta)
print(movie_rec_meta)

started add_movie_rating with user of 71568
movie_id  296
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  356
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  593
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  480
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  318
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  110
71568       5
get user recommendation started for 71568
movie_rec's user_data variable reset
started add_movie_rating with user of 71568
movie_id  457
71568       5
get user recommendatio

In [6]:
for index, row in movie_rec_meta_df.iterrows():

    movie_name = movie_rec.get_movie_title(row['movie_id'])
    similar_movie_names = ''
    for sim in row['similar_movie_ids']:
        similar_movie_names += movie_rec.get_movie_title(sim) + ', '
    # remove the last comma and space
    similar_movie_names = similar_movie_names[:-2]
    print(f"Movie: {movie_name}")
    print(f"Similar movies: {similar_movie_names}")

output_filename = 'data/metadata/movies_with_adjacent3_100.pkl'
movie_rec_meta_df.to_pickle(output_filename)

Movie: Pulp Fiction
Similar movies: Reservoir Dogs, Shawshank Redemption, The, Being John Malkovich
Movie: Forrest Gump
Similar movies: Singin' in the Rain, Message in a Bottle, Norbit
Movie: Silence of the Lambs, The
Similar movies: Pulp Fiction, Apollo 13, Batman
Movie: Jurassic Park
Similar movies: Dances with Wolves, Fugitive, The, Beauty and the Beast
Movie: Shawshank Redemption, The
Similar movies: Pulp Fiction, True Lies, Hotel Rwanda
Movie: Braveheart
Similar movies: Dances with Wolves, Jurassic Park, Sleepless in Seattle
Movie: Fugitive, The
Similar movies: Batman, Beauty and the Beast, Jurassic Park
Movie: Terminator 2: Judgment Day
Similar movies: Batman, Pulp Fiction, Ghost in the Shell (Kôkaku kidôtai)
Movie: Star Wars: Episode IV - A New Hope (a.k.a. Star Wars)
Similar movies: Star Wars: Episode I - The Phantom Menace, Wizard of Oz, The, Postman, The (Postino, Il)
Movie: Apollo 13
Similar movies: Pulp Fiction, Dances with Wolves, Toy Story
Movie: Batman
Similar movies: Te

In [4]:
similars_df = pd.read_pickle('data/metadata/movies_with_adjacent3_100.pkl')
# print(similars_df.head(5))

top_rated = pd.read_json('recs-app/public/top_rated_1000.json', )
top_rated = pd.merge(top_rated, similars_df, on='movie_id', how='left')
# top_rated.head(5)
top_rated.head(5)
top_rated.to_json('recs-app/public/top_rated_similars_1000.json', orient='records')

In [ ]:
movies = movie_rec.get_user_movies(u)
print(len(movies))
for m in movies:
    print(movie_rec.get_movie_title(m))

In [ ]:
ctr = 0
limit = 15
for idx, row in most_rated.head(10).iterrows():
    print(row['name'], idx)
    ctr += 1
    if ctr > limit:
        break

In [ ]:
# clean_titles = [movie_rec.get_movie_title(m) for m in set(movie_rec.ratings_df.index.values)]
import pandas as pd

# create a DataFrame containing the movie titles and ids
movies_df = pd.DataFrame({'title': movie_rec.movies_df['name']})

# count the occurrences of each title using the value_counts() method
title_counts = movies_df['title'].value_counts()

# filter out the titles that appear only once using boolean indexing
popular_titles = title_counts[title_counts > 1].index.tolist()

# get the corresponding movie ids for the popular titles
popular_movie_ids = movies_df.index.get_level_values('movie_id')[movies_df['title'].isin(popular_titles)].unique().tolist()

print(popular_movie_ids)

In [ ]:
# movie_rec.get_user_movies(69877)
for m in movie_rec.get_user_movies(69877):
    print(movie_rec.get_movie_title(m), movie_rec.get_user_movie_rating(69877, m))
# movie_rec.get_user_movie_rating(8546, 1)
# movie_rec.get_user_movie_rating(8546, 2)
# print(f'User {u} has seen...')
# for m in movie_rec.get_user_movies(u):
#     print(movie_rec.get_movie_title(m))

In [ ]:
users, dists = movie_rec.get_similar_users(u)
# for rec_user in users:
#     # print(f'{rec_user}\'s movies')
#     for m in movie_rec.get_user_movies(rec_user):
#         # print(movie_rec.get_movie_title(m))

# print(movie_rec.get_movie_title(20))
# print(movie_rec.get_movie_title(25))
# print(movie_rec.get_movie_title(50))

In [ ]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
# movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
# movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
# movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [ ]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)